In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from tqdm import tqdm
from random import shuffle

In [2]:
TRAIN_DIR = r'C:\Users\bartf\Downloads\FinalProject\train'      
TEST_DIR = r'C:\Users\bartf\Downloads\FinalProject\train'             
IMG_SIZE = 50               
LR = 1e-3                    

In [3]:
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR,'6conv-basic-video')         


In [4]:
def label_img(img):                   
    word_label = img.split('.')[-3]
    if word_label == 'cat' : return [1,0]
    elif word_label == 'dog': return [0,1]
    

In [5]:
def create_train_data():                           
    train_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        train_data.append([np.array(img),np.array(label)])
    shuffle(train_data)
    np.save('train_data.npy',train_data)
    return train_data

In [6]:
def process_test_data():                        
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        test_data.append([np.array(img),img_num])
    np.save('test_data.npy',test_data)
    return test_data


In [7]:
train_data = create_train_data()            

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [00:27<00:00, 906.21it/s]
G:\Anaconda\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [8]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tflearn.datasets.mnist as mnist

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir = 'log')       


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME) 
    print('model_loaded')

In [10]:
train = train_data[:-500]            
test = train_data[-500:]


In [11]:
X  = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE,1)
Y =  [i[1] for i in train]

test_x  = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE,1)
test_y =  [i[1] for i in test]

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=7, validation_set=({'input': test_x}, {'targets': test_y}),   
    snapshot_step=700, show_metric=True, run_id=MODEL_NAME)

Training Step: 330  | total loss: 0.63994 | time: 24.261s
| Adam | epoch: 001 | loss: 0.63994 - acc: 0.6305 -- iter: 21120/24500


In [ ]:
pred_y = model.predict(test_x)           

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random

orig = ""
pred = ""

for i in range(1,30):
    index = random.randint(0,499)
    if test_y[index][0]>test_y[index][1]:
        orig= 'cat'
    else:
        orig= 'dog'
    print("Actual Image depicts : "+ orig )
    if pred_y[index][0]>pred_y[index][1]:
        pred= 'cat'
    else:
        pred= 'dog'
    print("Model Predicted that the image is "+ pred +" with "+str(100*float("{:.2f}".format(max(pred_y[index][0],pred_y[index][1]))))+"% accuracy")
    plt.imshow( cv2.cvtColor(test_x[index],cv2.COLOR_GRAY2RGB)  )
    plt.show()
    print(57*'-')
    


